In [38]:
import pandas as pd
import xlrd
import numpy as np
from ipywidgets import *
from ipywidgets import interact,fixed

import matplotlib.pyplot as plt
pd.set_option('display.max_columns', 150)
pd.options.display.max_rows = 150

# read data from excel 

In [64]:
data = pd.read_excel('Daten_f_viesha.xlsx',sheet_name='DATENMITTAMB')
lookup = pd.read_excel('Daten_f_viesha.xlsx',sheet_name='lookup')
lookup['Kategorie'] = lookup.Kategorie.fillna(method='ffill')
data.describe()

,Sorte2,Grammatur,Tambour,Verhältnis HS in MB Einlage,Geschwindigkeit KM m/min,Zug SSW m/min,Zug 1.-2. Presse m/min,Zug 2.-3. Presse m/min,Zug 3. Presse - VTG m/min,BatchId,Form fraction 1,Form fraction 2,Form fraction 3,Form fraction 4,Form fraction 5,Curl %,Fines %,LenFrac 1,Lenfrac 2,Lenfrac 3,Lenfrac 4,Lenfrac 5,mittl. Faserlänge,mittl. Faserbreite,Kinks per fiber,Kinks per mm,Kinks segment length,Sampler status,Shive batchid,Shive fraction 1,Shive fraction 2,Shive fraction 3,"Shive amount, long","Shive amount, total",Shive weight,"Shive amount, wide",Width fraction 1,Width fraction 2,Width fraction 3,Width fraction 4,Width fraction 5,Tambournummer,Dicke,Flächengewicht,Spaltfestigkeit,Steifigkeit längs,Steifigkeit quer,Steifigkeit Geomittel,L/Q-Verhältnis
count,21694.000000,21694.000000,2.169400e+04,21694.000000,21694.000000,21694.000000,21694.000000,21694.000000,21694.000000,21584.000000,21583.00000,21583.000000,21583.00000,21584.000000,21584.000000,21584.000000,21583.000000,21583.000000,21583.000000,21584.000000,21583.000000,21583.000000,21583.000000,21583.000000,21583.000000,21583.000000,21584.000000,21584.000000,21584.000000,21583.000000,21583.000000,21583.000000,21583.000000,21583.000000,21583.000000,21584.000000,21583.000000,21583.000000,21583.000000,21583.000000,21583.000000,2.169400e+04,21686.000000,21694.000000,6931.000000,21694.000000,21694.000000,21694.000000,21694.000000
mean,1.944916,313.828017,4.122144e+06,74.263260,291.227682,11.468158,4.481627,2.766375,1.069213,31776.298554,10.91073,11.265541,11.86424,12.310049,13.287155,11.635908,26.433055,54.109224,30.441260,7.404955,5.879381,2.151234,0.674303,35.419525,0.152352,0.174870,0.763552,3.884266,31778.253892,6364.374878,503.088913,2040.269518,2027.249780,8276.514479,3.085133,886.611888,31.501327,32.848275,36.415700,42.532433,43.830114,4.011822e+10,538.316337,314.247780,93.019911,54.529088,22.774679,35.219926,2.377265
std,0.228957,58.974365,9.100553e+03,4.004359,62.079995,8.268474,6.038850,2.601720,6.145914,11583.973196,0.65806,0.610558,0.48571,0.646164,1.384189,0.471516,3.388142,3.165766,1.871928,1.004083,0.998036,0.643209,0.042070,1.714511,0.021063,0.021976,0.031209,5.287171,11581.982433,3606.826962,390.903517,455.886705,446.878276,2754.167761,0.939997,479.322748,1.212299,2.121690,1.885588,2.471556,4.697625,9.104191e+04,120.811241,59.167488,9.445230,30.698490,12.344930,19.426730,0.170942
min,0.000000,215.000000,4.107172e+06,47.466216,15.170402,-308.407056,-174.056700,-172.338618,-204.704339,0.000000,0.00000,0.000000,0.00000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,4.011807e+10,0.470000,215.500000,50.000000,15.350000,7.000000,10.942120,1.719780
25%,2.000000,275.000000,4.114338e+06,72.000000,247.835364,9.364958,3.565590,2.258376,0.916060,22007.500000,10.58000,10.930000,11.55000,11.880000,12.390000,11.370000,24.550000,52.020000,29.350000,6.810000,5.310000,1.710000,0.650000,34.400000,0.140000,0.160000,0.750000,1.000000,22009.500000,4722.000000,360.000000,1702.500000,1693.000000,6667.000000,2.470000,675.750000,31.110000,31.700000,35.180000,40.870000,40.720000,4.011814e+10,454.750000,272.500000,86.000000,31.800000,13.750000,21.149468,2.255459
50%,2.000000,300.000000,4.121208e+06,75.000000,307.665875,12.443871,4.595426,2.936239,1.148584,31172.000000,10.77000,11.140000,11.83000,12.280000,13.240000,11.560000,25.690000,53.790000,30.490000,7.440000,5.980000,2.130000,0.680000,35.100000,0.150000,0.180000,0.770000,3.000000,31174.000000,5426.000000,433.000000,1995.000000,1984.000000,7655.000000,2.910000,805.000000,31.560000,32.280000,36.280000,42.370000,43.610000,4.011821e+10,511.750000,300.750000,92.000000,43.200000,18.300000,28.349603,2.370829
75%,2.000000,350.000000,4.128308e+06,77.000000,342.035871,14.527341,5.184090,3.341152,1.302556,41575.000000,11.01000,11.42000

# null values?

In [13]:
data.isnull().sum()

Datum                              0
Sorte                              0
Sorte2                             0
Grammatur                          0
Tambour                            0
Verhältnis HS in MB Einlage        0
Geschwindigkeit KM m/min           0
Zug SSW m/min                      0
Zug 1.-2. Presse m/min             0
Zug 2.-3. Presse m/min             0
Zug 3. Presse - VTG m/min          0
BatchId                          110
Form fraction 1                  111
Form fraction 2                  111
Form fraction 3                  111
Form fraction 4                  110
Form fraction 5                  110
Curl %                           110
Fines %                          111
LenFrac 1                        111
Lenfrac 2                        111
Lenfrac 3                        110
Lenfrac 4                        111
Lenfrac 5                        111
mittl. Faserlänge                111
mittl. Faserbreite               111
Kinks per fiber                  111
K

# drop null values

In [ ]:
data = data.drop(['Feuchte','IGT','PPS'], axis=1)
lookup = lookup[~lookup.Parameter.isin(['Feuchte','IGT','PPS'])]

# Correlations?

In [79]:
def show_pearson_correlation(data=None,lookup=None, Sorte="41-1-2-100-275",Sorte2=2,Grammatur=275, target_col = 'Dicke'):
    df = data[data.Sorte==Sorte]
    df = df[df.Sorte2==Sorte2]
    df = df[df.Grammatur==Grammatur]
    # Now let us look at the correlation coefficient of each of these variables
    x_cols = [col for col in df.columns if col in lookup]
              #['Verhältnis HS in MB Einlage','Geschwindigkeit KM m/min','Zug SSW m/min',
              #'Zug 1.-2. Presse m/min','Zug 2.-3. Presse m/min ','Zug 3. Presse - VTG m/min']]

    labels = []
    values = []
    for col in x_cols:
        labels.append(col)
        values.append(np.corrcoef(df[col].values, df[target_col].values)[0,1])

    ind = np.arange(len(labels))
    width = 0.9
    fig, ax = plt.subplots(figsize=(10,10))
    plt.grid(True)
    rects = ax.barh(ind, np.array(values), color='y')
    ax.set_yticks(ind+((width)/2.))
    ax.set_yticklabels(labels, rotation='horizontal')
    ax.set_xlabel("Correlation coefficient")
    ax.set_title("Correlation coefficient")
    #autolabel(rects)
    plt.show()

# Korrelation Maschinendaten & Faserdaten

In [80]:
lookup_Maschine = lookup.Parameter[lookup.Kategorie=='Maschinenparameter'].values
interact(show_pearson_correlation
         , data = fixed(data)
         , lookup = fixed(lookup_Maschine)
         , Sorte=data.Sorte.unique()
         , Sorte2=data.Sorte2.unique()
         , Grammatur = data.Grammatur.unique()
         , target_col= lookup.Parameter[lookup.Kategorie=='Faserdaten']);

interactive(children=(Dropdown(description='Sorte', options=('41-1-2-100-275', '41-2-2-120-240', '41-2-2-105-2…

# Korrelation Maschinendaten-Laborergebnisse

In [81]:
interact(show_pearson_correlation
         , data = fixed(data)
         , lookup = fixed(lookup_Maschine)
         , Sorte=data.Sorte.unique()
         , Sorte2=data.Sorte2.unique()
         , Grammatur = data.Grammatur.unique()
         , target_col= lookup.Parameter[lookup.Kategorie=='Laborergebnisse']);

interactive(children=(Dropdown(description='Sorte', options=('41-1-2-100-275', '41-2-2-120-240', '41-2-2-105-2…

# Korrelation Faserdaten-Laborergebnisse

In [82]:
lookup_Faser = lookup.Parameter[lookup.Kategorie=='Faserdaten'].values
interact(show_pearson_correlation
         , data = fixed(data)
         , lookup = fixed(lookup_Faser)
         , Sorte=data.Sorte.unique()
         , Sorte2=data.Sorte2.unique()
         , Grammatur = data.Grammatur.unique()
         , target_col= lookup.Parameter[lookup.Kategorie=='Laborergebnisse']);

interactive(children=(Dropdown(description='Sorte', options=('41-1-2-100-275', '41-2-2-120-240', '41-2-2-105-2…